In [ ]:
import random


class Character:

    def __init__(self, name, max_hp, damage, armor, luck, max_mana):
        self.name = name
        self.max_hp = max_hp
        self.damage = damage
        self.armor = min(100, armor)
        self.luck = min(50, luck)
        self.max_mana = max_mana
        self.hp = max_hp
        self.mana = max_mana
        self.items = []
        self.potions = []

    def attack(self, character):
        dmg = self.damage
        dice = random.randint(1, 100)
        if dice <= self.luck:
            dmg = 2 * dmg
        return character.take_damage(dmg)

    def take_damage(self, damage):
        damage = damage * (1-self.armor/200)
        dice = random.randint(1, 100)
        if dice <= self.luck:
            damage = 0
        self.hp -= damage
        return damage

    def is_dead(self):
        return self.hp <= 0

    def heal(self, amount):
        healed = min(amount, self.mana, self.max_hp - self.hp)
        self.mana -= healed
        self.hp += healed
        return healed

    def add_item(self, item):
        if item.type == "p":
            self.potions.append(item)
            return
        if item.type == "o":
            self.damage += item.power
        elif item.type == "d":
            self.armor += item.power
            self.armor = min(100, self.armor)
        elif item.type == "l":
            self.luck += item.power
            self.luck = min(50, self.luck)
        self.items.append(item)

    def restore_mana(self):
        if len(self.potions) != 0:
            potion = self.potions.pop()
            restored = min(self.max_mana - self.mana, potion.power)
            self.mana += restored
            return restored
        return 0


class Item:

    def __init__(self, name, type, power):
        self.name = name
        self.type = type
        self.power = power


def main():

    characters = []
    for i in range(2):
        print("Make character ", i+1)
        name = input("Name?")
        max_hp = int(input("Maximum HP?"))
        damage = int(input("Damage?"))
        max_mana = int(input("Maximum mana?"))
        armor = int(input("Armor?"))
        luck = int(input("Luck?"))
        characters.append(Character(name, max_hp, damage, armor, luck, max_mana))
        print("")
    for i in range(2):
        print("Add items to character ", characters[i].name)
        while True:
            name = input("Name for your item? Press just enter to stop adding items")
            if name == "":
                print("")
                break
            i_type = input("Which type of weapon? Offensive (o), defensive (d), luck (l) or potion (p)?")
            power = int(input("What power is the item?"))
            item = Item(name, i_type, power)
            characters[i].add_item(item)
            print("")
    turn = 0
    while True:
        char = characters[turn]
        opponent = characters[(turn + 1) % 2]
        print("It is character ", char.name, "'s turn")
        print(char.name, ": ", char.hp, "hp,", opponent.name, ": ", opponent.hp, " hp")
        print(char.name, ": ", char.mana, "mana,", opponent.name, ": ", opponent.mana, " mana")
        move = input("What would you like to do? Attack (a), heal (h), use a manapotion (p)")
        if move == "a":
            dmg = char.attack(opponent)
            print("You deal ", dmg, "dmg")
        elif move == "h":
            print("How much do you want to heal? You currently have ", char.hp, " out of ", char.max_hp, " hp")
            amount = int(input("and " + str(char.mana) + " mana"))
            restored = char.heal(amount)
            print("You heal", restored, "HP")
        elif move == "p":
            restored = char.restore_mana()
            print("You restore", restored, "mana")
        turn = (turn+1) % 2
        print("")
        if opponent.is_dead():
            print(opponent.name, "is dead!", char.name, "wins!")
            break